In [ ]:
!pip install tflearn

In [ ]:
import pandas as pd
# modules we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# modules we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [ ]:
df = pd.read_excel('SampleDataset.xlsx')

In [ ]:
df

,Utterance,Label
0,What is plagiocephaly,introduction
1,What do you know about diabetes,introduction
2,General information about bruxism,introduction
3,Tell me what are Shingles,introduction
4,What is the cause of a stroke,causes
5,Causes of bruxism,causes
6,I want to know the causes of stroke,causes
7,What are the causes of Transitory Ischemia,causes
8,Tell me the causes of chlamydia,causes
9,causes of meningitis,causes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Utterance  27 non-null     object
 1   Label      27 non-null     object
dtypes: object(2)
memory usage: 560.0+ bytes


In [ ]:
df.describe()

,Utterance,Label
count,27,27
unique,27,6
top,What are the causes of Transitory Ischemia,causes
freq,1,6


In [ ]:
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')

def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stop:
            new_words.append(word)
    return new_words

def stem_words(words):
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def preprocess(sample):    
    # Tokenize
    words = nltk.word_tokenize(sample)

    words = remove_punctuation(words)
    words = to_lowercase(words)
    # lemmatize
    words = lemmatize_verbs(words)

    return words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df['text'] = df.Utterance.apply(preprocess)
df

,Utterance,Label,text
0,What is plagiocephaly,introduction,"[what, be, plagiocephaly]"
1,What do you know about diabetes,introduction,"[what, do, you, know, about, diabetes]"
2,General information about bruxism,introduction,"[general, information, about, bruxism]"
3,Tell me what are Shingles,introduction,"[tell, me, what, be, shingle]"
4,What is the cause of a stroke,causes,"[what, be, the, cause, of, a, stroke]"
5,Causes of bruxism,causes,"[cause, of, bruxism]"
6,I want to know the causes of stroke,causes,"[i, want, to, know, the, cause, of, stroke]"
7,What are the causes of Transitory Ischemia,causes,"[what, be, the, cause, of, transitory, ischemia]"
8,Tell me the causes of chlamydia,causes,"[tell, me, the, cause, of, chlamydia]"
9,causes of meningitis,causes,"[cause, of, meningitis]"


In [ ]:
df.Utterance[16]

'Do you have any information about the treatment of breast cancer?'

In [ ]:
df.text[16]

['do',
 'you',
 'have',
 'any',
 'information',
 'about',
 'the',
 'treatment',
 'of',
 'breast',
 'cancer']

In [ ]:
labels_list = df.Label
text_list = df.text
documents = [(df.text[i],df.Label[i]) for i in range(len(df.text))]
documents[16]

(['do',
  'you',
  'have',
  'any',
  'information',
  'about',
  'the',
  'treatment',
  'of',
  'breast',
  'cancer'],
 'treatment')

In [ ]:
unique=[]
for l in text_list:
  for w in l:
    unique.append(w)
words = sorted(list(set(unique)))
len(words)

74

In [ ]:
classes = sorted(list(set(labels_list.tolist())))

In [ ]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
6 classes ['causes', 'complications', 'diagnosis', 'introduction', 'prognosis', 'treatment']
74 unique stemmed words ['a', 'about', 'ache', 'acute', 'allergy', 'anorexia', 'any', 'associate', 'atherosclerotic', 'be', 'bladder', 'breast', 'bronchitis', 'bruxism', 'can', 'cancer', 'cardiosclerosis', 'cause', 'children', 'chlamydia', 'cholecystitis', 'complications', 'diabetes', 'diabetic', 'diagnose', 'die', 'do', 'enlargement', 'externa', 'for', 'from', 'general', 'have', 'headache', 'how', 'i', 'in', 'information', 'ischemia', 'ketoacidosis', 'know', 'me', 'meningitis', 'migraine', 'my', 'need', 'of', 'options', 'otitis', 'plagiocephaly', 'problems', 'prognosis', 'prostate', 'provide', 'relate', 's', 'shingle', 'show', 'stroke', 'syndrome', 'tell', 'the', 'there', 'think', 'to', 'tourette', 'transitory', 'treat', 'treatment', 'tummy', 'want', 'what', 'with', 'you']


In [ ]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle the features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True, validation_set=0.1) # 10% for validation
model.save('model.tflearn')

Training Step: 2999  | total loss: 1.09724 | time: 0.008s
| Adam | epoch: 1000 | loss: 1.09724 - acc: 0.8695 -- iter: 16/24
Training Step: 3000  | total loss: 1.02955 | time: 1.020s
| Adam | epoch: 1000 | loss: 1.02955 - acc: 0.7825 | val_loss: 2.08131 - val_acc: 0.0000 -- iter: 24/24
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
